In [4]:
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

from model import KeywordSpottingModel
from data_loader import load_speech_commands_dataset
from utils import set_memory_GB,print_model_size, log_to_file
from augmentations import add_time_shift_and_align
from train import trainig_loop

from torch.utils.data import DataLoader, Dataset
from librosa.feature import mfcc, delta
import numpy as np
import torch
import random


torch.cuda.is_available()
set_memory_GB(1)
train_ds, val_ds, test_ds, bg_noise_ds , info = load_speech_commands_dataset()

Memory fraction set to 0.022458079576498518
Memory fraction in GB: 1.0


2024-09-05 07:26:35.644869: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-05 07:26:35.652427: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-05 07:26:35.666839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 07:26:35.689944: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 07:26:35.696371: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attemptin

In [5]:
# # reproducibility
# seed = 42
# torch.manual_seed(seed)
# np.random.seed(seed)
# random.seed(seed)

In [6]:
# # load urbansound8k dataset
# import wget
# import os
# import zipfile
# import pandas as pd

# def download_urbansound8k():
#     if not os.path.exists('UrbanSound8K'):
#         os.mkdir('UrbanSound8K')
#         wget.download('https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz', 'UrbanSound8K/UrbanSound8K.tar.gz')
#         with zipfile.ZipFile('UrbanSound8K/UrbanSound8K.tar.gz', 'r') as zip_ref:
#             zip_ref.extractall('UrbanSound8K')
#         os.remove('UrbanSound8K/UrbanSound8K.tar.gz')
#     else:  
#         print("UrbanSound8K already exists")

# download_urbansound8k()


In [7]:

import os
import librosa
import numpy as np

# Path to the UrbanSound8K dataset
dataset_path = 'UrbanSound8K/UrbanSound8K/audio'

bgNoise = []
count = 0

# Loop through all folds and load audio files
for fold in range(1, 2):  # Folds are from fold1 to fold10
    fold_path = os.path.join(dataset_path, f'fold{fold}')
    
    # List all audio files in the current fold
    for file_name in os.listdir(fold_path):
        if file_name.endswith('.wav'):  # Process only .wav files
            file_path = os.path.join(fold_path, file_name)
            
            # Load the audio file
            audio, sample_rate = librosa.load(file_path, sr=None)
            # normalize audio
            audio = audio / np.max(np.abs(audio))
            bgNoise.append(audio)

In [29]:
# Define the dataset adapter:
class TFDatasetAdapter(Dataset):
    def __init__(self, tf_dataset, bg_noise_dataset, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentation=False, derivative=True, noise_level=0.3, MFCC_transform=True):
        self.tf_dataset = tf_dataset
        self.data = list(tf_dataset)
        self.bg_noise_data = list(bg_noise_dataset) if bg_noise_dataset is not None else None
        self.fixed_length = fixed_length
        self.n_mfcc = n_mfcc
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.augmentation = augmentation
        self.derivative = derivative
        self.noise_level = noise_level
        self.MFCC_transform = MFCC_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio, label = self.data[idx]
        audio = audio.numpy()

        # Normalize the audio tensor
        audio = audio / np.max(np.abs(audio))

        # Convert to float
        audio = audio.astype(np.float32)

        # Ensure the audio tensor has the correct shape (1D array)
        if audio.ndim > 1:
            audio = np.squeeze(audio)


        # Add noise from bg_noise data
        if self.bg_noise_data:
            bg_noise_audio = random.choice(self.bg_noise_data)
            # bg_noise_audio = bg_noise_audio.numpy().astype(np.float32)

            # Trim or pad bg_noise to match the audio length
            if len(bg_noise_audio) < len(audio):
                bg_noise_audio = np.pad(bg_noise_audio, (0, len(audio) - len(bg_noise_audio)), mode='constant')
            else:
                # Take a random slice of bg_noise_audio with the same length as the original audio
                start_idx = random.randint(0, len(bg_noise_audio) - len(audio))
                bg_noise_audio = bg_noise_audio[start_idx:start_idx + len(audio)]

            # Add bg_noise as noise to the original audio
            audio = audio + self.noise_level * bg_noise_audio

        # Pad or trim the audio to the fixed length
        if len(audio) < self.fixed_length:
            audio = np.pad(audio, (0, self.fixed_length - len(audio)), mode='constant')
        else:
            audio = audio[:self.fixed_length]
        output = audio
            

        # Apply augmentations if any
        if self.augmentation:
            for aug in self.augmentation:
                audio = aug(audio)


        if self.MFCC_transform:
        # Create MFCCs from an audio tensor using Librosa.
            audio = audio.astype(np.float32)
            MFCC = mfcc(y=audio, sr=16000, n_mfcc=self.n_mfcc, n_fft=self.n_fft, hop_length=self.hop_length, n_mels=self.n_mels)
            if self.derivative:
                # Create MFCC second, first order delta
                MFCC_delta = delta(MFCC)
                MFCC_delta2 = delta(MFCC, order=2)

                # Stack the three MFCCs together
                MFCC = np.vstack([MFCC, MFCC_delta, MFCC_delta2])

            # Remove extra dimension if it exists
            if output.ndim == 3:
                MFCC = MFCC.squeeze(-1)

            output = MFCC

        return torch.tensor(output, dtype=torch.float32), torch.tensor(label.numpy(), dtype=torch.long)    





In [30]:
augmentations = [
    lambda x: add_time_shift_and_align(x),
]
# Convert the TFDS dataset to a PyTorch Dataset
fixed_length = 16000
n_mfcc = 13
n_fft = 640
hop_length = 320
n_mels = 40
pytorch_train_dataset = TFDatasetAdapter(train_ds.take(10), bgNoise, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentation = augmentations, noise_level=1.0, MFCC_transform=False)
# pytorch_val_dataset = TFDatasetAdapter(val_ds, None, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentation=None)
# Create a DataLoader to feed the data into the model
batch_size = 32
train_loader = DataLoader(pytorch_train_dataset, batch_size=batch_size, shuffle=True,num_workers=4,prefetch_factor=2)
# val_loader = DataLoader(pytorch_val_dataset, batch_size=batch_size, shuffle=False,num_workers=4,prefetch_factor=2)

In [31]:
import IPython.display as ipd
for audio,label in pytorch_train_dataset:
    print(label.numpy())
    ipd.display(ipd.Audio(audio.numpy(), rate=16000))



4


5


7


5


7


2


7


6


6


6


In [28]:
import IPython.display as ipd
for audio,label in train_ds.take(10):
    print(label.numpy())
    ipd.display(ipd.Audio(audio.numpy(), rate=16000))



4


5


7


5


7


...

7


6


6


6


2024-09-05 07:31:30.125439: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
